In [1]:
import pandas as pd

In [81]:
df=pd.read_csv("train.csv")

In [85]:
df[df["id"]==1]

,id,title,author,text,label
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [9]:
df=df.dropna()
x=df.loc[:,df.columns!="labels"]

In [10]:
y=df["label"]

In [11]:
x.shape

(18285, 5)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf


In [14]:
tf.__version__

'2.7.0'

In [41]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout

In [16]:
voc_size=10000

In [17]:
messages=x.copy()

In [1]:
messages
messages.reset_index(inplace=True)

NameError: name 'messages' is not defined

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
from nltk.stem import WordNetLemmatizer

In [23]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
lm = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [lm.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [29]:
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay 

In [31]:
onehot=[one_hot(words,voc_size)for words in corpus] 
onehot

[[7806, 3680, 1620, 4839, 4300, 3683, 4824, 5511, 981, 9306],
 [2981, 7951, 6071, 555, 734, 5658, 4205],
 [3116, 8062, 4299, 5590],
 [3534, 7230, 3516, 8347, 737, 2407],
 [7923, 734, 6587, 7349, 259, 2804, 734, 3759, 9651, 5479],
 [9959,
  4038,
  9650,
  3353,
  3369,
  6978,
  4341,
  9827,
  4819,
  1911,
  7035,
  5957,
  9615,
  6527,
  4205],
 [477, 6675, 6233, 9776, 6326, 9849, 2414, 7880, 1583, 1972, 3919],
 [3587, 2200, 956, 9466, 5417, 1970, 6978, 7430, 1583, 1972, 3919],
 [3717, 2143, 6511, 5943, 5689, 6632, 8814, 824, 6978, 7630],
 [6771, 9298, 1433, 593, 6904, 6023, 5075, 2231],
 [7111, 9996, 1862, 9902, 5186, 7938, 7247, 8651, 9884, 2994, 7778],
 [8347, 3565, 4300, 882, 6978, 5417],
 [3123, 9815, 4977, 8796, 5312, 9245, 7439, 7074, 9125],
 [2000, 1968, 1436, 3861, 1209, 3151, 2700, 1583, 1972, 3919],
 [1065, 7913, 1132, 3726, 6952, 1583, 1972, 3919],
 [1747, 3170, 2255, 2729, 9124, 8099, 4780, 8801, 9423, 2844],
 [7679, 5166, 7951],
 [3583, 9324, 8424, 8562, 6978, 6872, 4

In [33]:
corpus[1]

'flynn hillary clinton big woman campus breitbart'

In [34]:
onehot[1]

[2981, 7951, 6071, 555, 734, 5658, 4205]

In [37]:
length=30
pad_length=pad_sequences(onehot,padding="post",maxlen=length)

In [38]:
pad_length.shape

(18285, 30)

In [42]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [44]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 40)            400000    
                                                                 
 dropout (Dropout)           (None, 30, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [45]:
pad_length.shape

(18285, 30)

In [46]:
y.shape

(18285,)

In [48]:
import numpy as np

In [49]:
x_transformed=np.array(pad_length)

In [50]:
x_transformed

array([[7806, 3680, 1620, ...,    0,    0,    0],
       [2981, 7951, 6071, ...,    0,    0,    0],
       [3116, 8062, 4299, ...,    0,    0,    0],
       ...,
       [2620, 4779, 3897, ...,    0,    0,    0],
       [1844, 5417,  286, ...,    0,    0,    0],
       [6175, 1456, 6620, ...,    0,    0,    0]])

In [51]:
x_transformed.shape

(18285, 30)

In [52]:
y_transformed=np.array(y)

In [53]:
y_transformed

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_transformed, y_transformed, test_size=0.33, random_state=42)

In [55]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 20s 82ms/step - loss: 0.3525 - accuracy: 0.8224 - val_loss: 0.2029 - val_accuracy: 0.9181
Epoch 2/10
192/192 [==============================] - 13s 67ms/step - loss: 0.1394 - accuracy: 0.9503 - val_loss: 0.2048 - val_accuracy: 0.9147
Epoch 3/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0849 - accuracy: 0.9717 - val_loss: 0.2435 - val_accuracy: 0.9163
Epoch 4/10
192/192 [==============================] - 12s 65ms/step - loss: 0.0603 - accuracy: 0.9801 - val_loss: 0.2693 - val_accuracy: 0.9080
Epoch 5/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0417 - accuracy: 0.9866 - val_loss: 0.3099 - val_accuracy: 0.9132
Epoch 6/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0370 - accuracy: 0.9895 - val_loss: 0.3992 - val_accuracy: 0.9049
Epoch 7/10
192/192 [==============================] - 13s 66ms/step - loss: 0.0236 - accuracy: 0.9922 - val_loss: 0.3813 - val_accuracy:

In [56]:
y_pred=model.predict(X_test)

In [57]:
y_pred=np.where(y_pred > 0.6, 1,0) 

In [58]:
from sklearn.metrics import confusion_matrix

In [59]:
confusion_matrix(y_test,y_pred)

array([[3156,  263],
       [ 321, 2295]], dtype=int64)

In [60]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9032311516155758

#### Predicting using the test data

In [86]:
df2=pd.read_csv("test.csv")

In [87]:
df2.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [89]:
df2[df2["id"]==20800]

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."


In [90]:
df[df["author"]=="David Streitfeld"]

,id,title,author,text,label
5371,5371,"For Fact-Checking Website Snopes, a Bigger Rol...",David Streitfeld,SAN DIEGO — The last line of defense agains...,0
8724,8724,Tech Opposition to Trump Propelled by Employee...,David Streitfeld,"In late September, a group of tech leaders sta...",0
10376,10376,Google Silences Doubters With Blockbuster Quar...,David Streitfeld,SAN FRANCISCO — It’s good to be Google. Som...,0
12288,12288,"‘I’m Here to Help,’ Trump Tells Tech Executive...",David Streitfeld,The meeting between Donald J. Trump and the ...,0
13944,13944,Will the New Apple iPhone Have a Headphone Jac...,David Streitfeld,"SAN FRANCISCO — If Apple had its way, this ...",0
16501,16501,Activism Hits Even the Less Flashy Tech Compan...,David Streitfeld,SAN FRANCISCO — As soon as President Trump ...,0
18963,18963,Peter Thiel to Donate $1.25 Million in Support...,David Streitfeld,"SAN FRANCISCO — Peter Thiel, true to his re...",0
20341,20341,Amazon Is Quietly Eliminating List Prices - Th...,David Streitfeld,SAN FRANCISCO — In a major shift for online...,0
20356,20356,Silicon Valley Finds Trump’s Disruption Unwelc...,David Streitfeld,"PALO ALTO, Calif. — Donald J. Trump would n...",0
